In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/information-retrieval-agent/wikipedia_dataset (6).csv
/kaggle/input/information-retrieval-agent/en.tsv


In [2]:
import pandas as pd
import pickle
!pip install wikipedia
!pip install sentence_transformers
import wikipedia
from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter, defaultdict
from scipy.stats import entropy
from itertools import combinations
import ast

# Download necessary NLTK data for tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# 1. Preprocessing Helper Function
def preprocess_documents(documents):
    """Ensure all documents are valid strings and handle invalid data."""
    if not isinstance(documents, list):
        raise ValueError("Input documents must be a list.")
    return [doc if isinstance(doc, str) else "" for doc in documents]

# 2. Lexical Diversity Calculation
def calculate_lexical_diversity(documents):
    """
    Calculate the lexical diversity of a collection of documents.

    Args:
        documents (list of str): The list of documents (strings) to be analyzed.

    Returns:
        float: The ratio of unique words to total words in the documents.
    """
    total_words = sum(len(nltk.word_tokenize(doc)) for doc in documents if doc.strip())
    unique_words = len(set(word.lower() for doc in documents for word in nltk.word_tokenize(doc) if doc.strip()))
    return unique_words / total_words if total_words > 0 else 0
# 3. Semantic-Based Diversity Calculation
def calculate_semantic_diversity(embeddings):
    if len(embeddings) < 2:
        return 0.0  # Return a float, not a dictionary, if there are fewer than 2 embeddings

    # Calculate cosine similarity between embeddings
    cosine_sim = cosine_similarity(embeddings)
    
    # Get the upper triangle matrix indices (pairs of embeddings)
    avg_similarity = np.mean(cosine_sim[np.triu_indices_from(cosine_sim, k=1)])

    # Return 1 - avg_similarity as the semantic diversity score (as a float)
    return 1 - avg_similarity  # This is a float value

# 4. Category-Based Diversity Calculation
def calculate_category_diversity(articles):
    """
    Calculate diversity based on the intersection and coverage of categories in the dataset.
    
    Args:
        articles (list of lists): A list of lists where each inner list represents categories for an article.

    Returns:
        float: Category diversity score.
    """
    # Coverage: Average number of categories per article
    avg_coverage = np.mean([len(categories) for categories in articles])

    # Pairwise Overlap Calculation: Measures the overlap of categories between pairs of articles
    overlap_sum = 0
    article_pairs = list(combinations(articles, 2))
    for cat1, cat2 in article_pairs:
        set1, set2 = set(cat1), set(cat2)
        intersection = len(set1 & set2)
        union = len(set1 | set2)
        overlap_sum += intersection / union if union else 0

    # Global Diversity: Measures how diverse the dataset is based on category overlap
    max_possible_overlap = len(article_pairs)
    diversity_score = 1 - (overlap_sum / max_possible_overlap) if max_possible_overlap > 0 else 0
    return diversity_score

# 4. Combine Metrics into a Diversity Score Calculation
def calculate_diversity_score(submission, weights=None):
    # Extract content and categories from the dataset
    documents = [page['content'] for page in submission]
    categories = [page['categories'] for page in submission]
    embeddings = [page['embeddings'] for page in submission]

    if weights is None:
        weights = {'lexical': 0.3, 'semantic': 0.4, 'category': 0.3}

    # Lexical Diversity
    lexical_diversity = calculate_lexical_diversity(documents)

    # Semantic Diversity
    semantic_diversity = calculate_semantic_diversity(embeddings)

    # Category Diversity
    category_diversity = calculate_category_diversity(categories)

    # Overall Diversity Score (weighted sum of the three metrics)
    diversity_score = (
        weights['lexical'] * lexical_diversity +
        weights['semantic'] * semantic_diversity +
        weights['category'] * category_diversity
    )

    return {
        'Lexical Diversity': lexical_diversity,
        'Semantic Diversity': semantic_diversity,
        'Category Diversity': category_diversity,
        'Overall Diversity Score': diversity_score
    }

def get_dataset_column(dataset,columns_name):
    column = []
    for i in dataset:
        column.append(i[columns_name])
    return column
# 5. WikiRank Score Calculation
def get_wikirank_score(dataset, wikirank_df):
    """
    Calculate the mean WikiRank score for a given dataset, ensuring all titles are present in the WikiRank dataset.

    Args:
        dataset (pd.DataFrame): The dataset containing a 'title' column.
        wikirank_df (pd.DataFrame): The DataFrame containing 'page_name' and 'wikirank_quality'.

    Returns:
        float: The mean WikiRank score for the dataset.

    Raises:
        ValueError: If any titles in the dataset are not found in wikirank_df['page_name'].
    """
    dataset = pd.DataFrame({'title':get_dataset_column(dataset,'title')})
    # Check if all titles in the dataset are present in the WikiRank dataset
    missing_titles = set(dataset['title']) - set(wikirank_df['page_name'])
    if missing_titles:
        raise ValueError(f"The following titles are missing from wikirank_df['page_name']: {missing_titles}")

    # Merge the datasets to calculate the mean WikiRank score
    merged_df = dataset.merge(wikirank_df, left_on='title', right_on='page_name', how='inner')

    # Extract the WikiRank quality scores and calculate the mean score
    scores = merged_df['wikirank_quality']
    return scores.mean()
class WikipediaAPI:
    def __init__(self, page_request_limit=6500, wikirank_datasets_with_quality_scores_en_tsv='/kaggle/input/information-retrieval-agent/en.tsv'):
        self.wikirank_df = pd.read_csv(wikirank_datasets_with_quality_scores_en_tsv, sep='\t')
        self.legal_pages = self.wikirank_df['page_name'].tolist()
        self.page_request_limit = page_request_limit
        self.list_of_known_pages = []
        self.page_requests_used = 0
        self.fetched_pages = []
        self.model = SentenceTransformer('all-MiniLM-L6-v2')  # Initialize embedding model
        self.dataset = []
        self.visited_disambiguations = set()  # Track visited disambiguation pages
        self.max_disambiguation_retries = 5  # Max retries to avoid infinite loop

    def _increment_request(self):
        if self.page_requests_used >= self.page_request_limit:
            raise ValueError("API request limit exceeded. No further requests are allowed.")
        self.page_requests_used += 1

    def _check_legal_request(self, page_name):
        if page_name not in self.list_of_known_pages:
            raise ValueError(f"You are applying illegal request: {page_name} is not known to you")
        if page_name not in self.legal_pages:
            self.page_requests_used -= 1
            raise ValueError(f"Page {page_name} is not in the list of accessed pages. You cannot retrieve its data.")
    
    def search_pages(self, query):
        self._increment_request()
        try:
            page_names = wikipedia.search(query, results=10)
            if page_names:  # Ensure that the search returned results
                self.list_of_known_pages.extend(page_names)
                return page_names
            else:
                print(f"No pages found for query '{query}'")  # Inform the user if no results are found
                return []
        except Exception as e:
            print(f"Search failed for query '{query}': {e}")
            return []

    
    def fetch_page(self, page_name, retry_count=0):
        self._increment_request()

        # First, check if the page is legal before attempting to fetch it
        try:
            self._check_legal_request(page_name)
        except ValueError as e:
            print(str(e))  # Print the error and skip the page
            return None

        try:
            # Attempt to fetch the page
            page = wikipedia.page(page_name)
            page_info = {
                'title': page.title,
                'content': page.content,
                'url': page.url,
                'links': page.links,
                'categories': page.categories,
                'embeddings': self.model.encode(page.content)  # Add embeddings when fetching the page
            }
            self.fetched_pages.append(page_info)
            return page_info
        except wikipedia.exceptions.DisambiguationError as e:
            # Avoid looping on the same disambiguated page
            print(f"Disambiguation error for page '{page_name}': {e.options}")
            # Filter out pages we've already visited during disambiguation
            unvisited_options = [opt for opt in e.options if opt not in self.visited_disambiguations]
        
            if unvisited_options:
                # Choose the first unvisited disambiguation option
                chosen_page = unvisited_options[0]
                print(f"Choosing '{chosen_page}' from disambiguation options.")
                self.visited_disambiguations.add(chosen_page)
                if retry_count < self.max_disambiguation_retries:
                    return self.fetch_page(chosen_page, retry_count + 1)  # Retry with the new page
                else:
                    print(f"Max retries reached for disambiguation of '{page_name}'")
                    return None  # Max retries exceeded
            else:
                print(f"No unvisited disambiguation options left for '{page_name}'")
                return None  # No new disambiguation options to try
        except Exception as e:
            print(f"Failed to fetch page '{page_name}': {e}")
            return None


    def save_page(self, page_name):
        page = next((page for page in self.fetched_pages if page['title'] == page_name), None)
        if page:
            self.dataset.append(page)
            print(f"Page '{page_name}' saved with embeddings.")
        else:
            print(f"Page '{page_name}' not found or skipped due to disambiguation.")
        
    def Calculate_embeddings(self):
        for i in range(len(self.dataset)):
            content= self.dataset[i]['content']
            self.dataset[i]['embeddings'] = self.model.encode(content)
        print("Embeddings calculated")
    
    def save_dataset(self, pkl_path, scores_csv_path):
        # Ensure embeddings are calculated for all pages before saving
        for page in self.dataset:
            if 'embeddings' not in page:
                page['embeddings'] = self.model.encode(page['content'])

        with open(pkl_path, 'wb') as f:
            pickle.dump(self.dataset, f)

        # Calculate diversity score
        lexical_diversity = calculate_lexical_diversity([page['content'] for page in self.dataset])
        semantic_diversity = calculate_semantic_diversity([self.model.encode(page['content']) for page in self.dataset])
        category_diversity = calculate_category_diversity([page['categories'] for page in self.dataset])

        # Combine the three diversity metrics into an overall score
        diversity_score = (0.3 * lexical_diversity + 0.3 * category_diversity + 0.4 * semantic_diversity)

        # Calculate WikiRank score
        wikirank_score = get_wikirank_score(self.dataset, self.wikirank_df)

        # Final Score Calculation
        final_score = (diversity_score * 100 + wikirank_score) / 2

        scores = {
            'Dataset Size': len(self.dataset),
            'WikiRank Score': wikirank_score,
            'Diversity Score': diversity_score,
            'Final Score': final_score
        }
        scores_df = pd.DataFrame([scores])
        scores_df.to_csv(scores_csv_path, index=False)
        print(f"Scores saved to {scores_csv_path}")

    def is_legal_page(self, page_name):
        return page_name in self.legal_pages

    def get_usage_summary(self):
        return {
            "page_requests_used": self.page_requests_used,
            "page_request_limit": self.page_request_limit,
            "list_of_known_pages": self.list_of_known_pages
        }


In [4]:
import random
global api

explored_pages = {}

def agent_run():
    global api
    api = WikipediaAPI() 
    
    # Seed queries
    seed_queries = ["science", "history", "technology", "medicine",
        "quantum mechanics", "neuroscience", "deep learning algorithms", 
        "ancient civilizations", "medieval philosophy", "digital marketing", 
        "neuroscience", "geopolitics", "climate change",
        "renaissance art", "modernist literature", "computer vision",
        "ancient civilizations", "ethics", "recent",
        "popular", "biology", "engineering", "sociology", 
        "ancient civilizations", "marketing", 
        "blockchain", "AI", "healthcare", "machine learning", 
        "renaissance art", "space exploration", "world war", 
        "sustainable development", "quantum physics"
        ]
    
    explored_topics = set(seed_queries)

    # Fetch initial pages based on seed queries
    for query in seed_queries:
        pages = api.search_pages(query)
        for page in pages:
            # save it
            page_info = api.fetch_page(page)
            if page_info:
                # Calculate lexical diversity per page
                lexical_diversity = calculate_lexical_diversity(page_info['content'])
                explored_pages[page] = lexical_diversity 
                if len(api.dataset) < 5000: 
                    api.save_page(page_info['title'])

    # Explore based on lexical diversity
    max_additional_pages = 5000 - len(api.dataset)
    iterations = 0
    diversity_threshold = 0.04

    while len(api.dataset) < 5000 and iterations < max_additional_pages:
        iterations += 1

        # pages in the list to explore
        if not api.list_of_known_pages:
            print("No known pages to explore.")
            break
        
        # Sort pages by lexical diversity score in descending order
        sorted_pages = sorted(explored_pages.items(), key=lambda x: x[1], reverse=True)

        page_fetched = False
        for page, diversity_score in sorted_pages:
            if page not in explored_topics and len(api.dataset) < 5000:
                print(f"Exploring page: {page} with lexical diversity score: {diversity_score}")
                
                # save it if valid
                page_info = api.fetch_page(page)
                if page_info:
                    if len(api.dataset) < 5000:
                        api.save_page(page_info['title'])
                        explored_topics.add(page) 
                        page_fetched = True
                        break 

        # If no page fetched, fall back to lower-diversity pages
        if not page_fetched:
            print("No high-diversity page found. Adding page with lower diversity score.")
            for page, diversity_score in sorted_pages:
                if page not in explored_topics and diversity_score < diversity_threshold:
                    page_info = api.fetch_page(page)
                    if page_info:
                        if len(api.dataset) < 5000:
                            api.save_page(page_info['title'])
                            explored_topics.add(page)
                            break
        
        # Break the loop once 5000 pages are saved
        if len(api.dataset) >= 5000:
            print("Target of 5000 pages reached.")
            break

        # Check if API calls are nearing the limit (6500 calls)
        if api.page_requests_used >= 6500:
            print("API call limit reached. Stopping agent.")
            break
    
    # Save dataset
    api.save_dataset("final_dataset6.pkl", "final_scores6.csv")

# Run the agent
agent_run()



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = B

Disambiguation error for page 'Science': ['Sincé', 'Sińce', 'Since (film)', 'Since (album)', 'Since (rapper)']
Choosing 'Sincé' from disambiguation options.
You are applying illegal request: Sincé is not known to you
Disambiguation error for page 'Science (disambiguation)': ['computer science', 'Computer Sciences Corporation', 'Computer Science (journal)', 'Computer Science (UIL)']
Choosing 'computer science' from disambiguation options.
You are applying illegal request: computer science is not known to you


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Page 'Computer science' saved with embeddings.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Scores saved to final_scores6.csv


In [5]:
diversity_scores = calculate_diversity_score(api.dataset)

# Print individual diversity scores and the overall diversity score
print(f"Lexical Diversity: {diversity_scores['Lexical Diversity']}")
print(f"Semantic Diversity: {diversity_scores['Semantic Diversity']}")
print(f"Category Diversity: {diversity_scores['Category Diversity']}")
print(f"Overall Diversity Score: {diversity_scores['Overall Diversity Score']}")


Lexical Diversity: 0.019217591958131316
Semantic Diversity: 0.8940574750304222
Category Diversity: 0.9152433655585269
Overall Diversity Score: 0.6379612772671663
